In [5]:
import numpy as np
import matplotlib.pyplot as plt

import glob

%matplotlib inline

In [8]:
forest_data_path = glob.glob("data/forest/*")
tallbuilding_data_path = glob.glob("data/tallbuilding/*")
highway_data_path = glob.glob("data/highway/*")

In [7]:
temp1 = np.loadtxt(forest_data_path[0])
temp2 = np.loadtxt(tallbuilding_data_path[0])

In [4]:
print(temp1.shape, temp2.shape)

(36, 23) (36, 23)


In [13]:
X = []
Y = []
for class_path,label in zip([forest_data_path, tallbuilding_data_path, highway_data_path],[1,2,3]:
    

array([  1.00660000e-01,   3.99470000e-01,   4.37360000e-01,
         1.36450000e-03,   5.85230000e-05,   4.36820000e-04,
        -8.94020000e-01,   5.97820000e-02,   2.02170000e-01,
         6.29910000e-04,   1.03050000e-02,   4.78730000e-03,
         1.46140000e-03,   4.53540000e-04,   8.99510000e-03,
         6.17310000e-03,   1.16410000e-02,   3.32830000e+00,
         3.19940000e+00,   3.30230000e+00,   3.25160000e+00,
         3.37700000e+00,   3.02360000e+00])

In [9]:
from sklearn.datasets import load_iris
data = load_iris()

In [16]:
X_train = data.target

In [17]:
X_train.shape

(150,)

In [18]:
temp = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [20]:
temp = temp.reshape(1,-1)

In [21]:
temp

array([[1, 2, 3, 4, 5, 6, 7, 8, 9]])